https://pytorch.org/tutorials/recipes/recipes/profiler_recipe.html

In [1]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline


# PyTorch Profiler
This recipe explains how to use PyTorch profiler and measure the time and
memory consumption of the model's operators.

## Introduction
PyTorch includes a simple profiler API that is useful when user needs
to determine the most expensive operators in the model.

In this recipe, we will use a simple Resnet model to demonstrate how to
use profiler to analyze model performance.

## Setup
To install ``torch`` and ``torchvision`` use the following command:

```sh
pip install torch torchvision
```


## Steps

1. Import all necessary libraries
2. Instantiate a simple Resnet model
3. Using profiler to analyze execution time
4. Using profiler to analyze memory consumption
5. Using tracing functionality
6. Examining stack traces
7. Visualizing data as a flame graph
8. Using profiler to analyze long-running jobs

### 1. Import all necessary libraries

In this recipe we will use ``torch``, ``torchvision.models``
and ``profiler`` modules:




In [2]:
import torch
import torchvision.models as models
from torch.profiler import profile, record_function, ProfilerActivity

/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


### 2. Instantiate a simple Resnet model

Let's create an instance of a Resnet model and prepare an input
for it:




In [3]:
model = models.resnet18()
inputs = torch.randn(5, 3, 224, 224)

### 3. Using profiler to analyze execution time

PyTorch profiler is enabled through the context manager and accepts
a number of parameters, some of the most useful are:

- ``activities`` - a list of activities to profile:
   - ``ProfilerActivity.CPU`` - PyTorch operators, TorchScript functions and
     user-defined code labels (see ``record_function`` below);
   - ``ProfilerActivity.CUDA`` - on-device CUDA kernels;
- ``record_shapes`` - whether to record shapes of the operator inputs;
- ``profile_memory`` - whether to report amount of memory consumed by
  model's Tensors;
- ``use_cuda`` - whether to measure execution time of CUDA kernels.

Note: when using CUDA, profiler also shows the runtime CUDA events
occurring on the host.



Let's see how we can use profiler to analyze the execution time:



In [4]:
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
        model(inputs)

Note that we can use ``record_function`` context manager to label
arbitrary code ranges with user provided names
(``model_inference`` is used as a label in the example above).

Profiler allows one to check which operators were called during the
execution of a code range wrapped with a profiler context manager.
If multiple profiler ranges are active at the same time (e.g. in
parallel PyTorch threads), each profiling context manager tracks only
the operators of its corresponding range.
Profiler also automatically profiles the asynchronous tasks launched
with ``torch.jit._fork`` and (in case of a backward pass)
the backward pass operators launched with ``backward()`` call.

Let's print out the stats for the execution above:



In [5]:
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                  model_inference         4.60%       4.221ms       100.00%      91.835ms      91.835ms             1  
                     aten::conv2d         0.40%     366.856us        73.66%      67.650ms       3.382ms            20  
                aten::convolution         0.35%     318.362us        73.27%      67.283ms       3.364ms            20  
               aten::_convolution         0.24%     219.702us        72.92%      66.964ms       3.348ms            20  
         aten::mkldnn_convolution        72.34%      66.429ms        72.68%      66.745ms       3.337ms            20  
                 aten::max_pool2d       

Here we see that, as expected, most of the time is spent in convolution (and specifically in ``mkldnn_convolution``
for PyTorch compiled with ``MKL-DNN`` support).
Note the difference between self cpu time and cpu time - operators can call other operators, self cpu time excludes time
spent in children operator calls, while total cpu time includes it. You can choose to sort by the self cpu time by passing
``sort_by="self_cpu_time_total"`` into the ``table`` call.

To get a finer granularity of results and include operator input shapes, pass ``group_by_input_shape=True``
(note: this requires running the profiler with ``record_shapes=True``):



In [6]:
print(prof.key_averages(group_by_input_shape=True).table(sort_by="cpu_time_total", row_limit=10))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls                                                                      Input Shapes  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  
                  model_inference         4.60%       4.221ms       100.00%      91.835ms      91.835ms             1                                                                                []  
                     aten::conv2d         0.03%      25.682us        16.07%      14.756ms       3.689ms             4                             [[5, 64, 56, 56], [64, 64, 3, 3], [], [], [], 

Note the occurrence of ``aten::convolution`` twice with different input shapes.



Profiler can also be used to analyze performance of models executed on GPUs:



In [7]:
model = models.resnet18().cuda()
inputs = torch.randn(5, 3, 224, 224).cuda()

with profile(activities=[
        ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True) as prof:
    with record_function("model_inference"):
        model(inputs)

print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        model_inference         0.00%       0.000us         0.00%       0.000us       0.000us      77.541ms        94.99%      77.541ms      38.771ms             2  
                                        model_inference         1.01%       1.806ms       100.00%     178.994ms     178.994ms       0.000us         0.00%       4.093ms       4.093ms             1  
         

(Note: the first use of CUDA profiling may bring an extra overhead.)



Note the occurrence of on-device kernels in the output (e.g. ``sgemm_32x32x32_NN``).



### 4. Using profiler to analyze memory consumption

PyTorch profiler can also show the amount of memory (used by the model's tensors)
that was allocated (or released) during the execution of the model's operators.
In the output below, 'self' memory corresponds to the memory allocated (released)
by the operator, excluding the children calls to the other operators.
To enable memory profiling functionality pass ``profile_memory=True``.



In [8]:
model = models.resnet18()
inputs = torch.randn(5, 3, 224, 224)

with profile(activities=[ProfilerActivity.CPU],
        profile_memory=True, record_shapes=True) as prof:
    model(inputs)

print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=10))

print(prof.key_averages().table(sort_by="cpu_memory_usage", row_limit=10))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                      aten::empty         0.47%     330.637us         0.47%     330.637us       1.653us      94.86 Mb      94.86 Mb           200  
    aten::max_pool2d_with_indices         9.30%       6.598ms         9.30%       6.598ms       6.598ms      11.48 Mb      11.48 Mb             1  
                      aten::addmm         0.17%     118.586us         0.18%     127.930us     127.930us      19.53 Kb      19.53 Kb             1  
                       aten::mean         0.02%      15.597us         0.11%      74.759us      74.759us      10.

### 5. Using tracing functionality

Profiling results can be outputted as a ``.json`` trace file:



In [9]:
model = models.resnet18().cuda()
inputs = torch.randn(5, 3, 224, 224).cuda()

with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA]) as prof:
    model(inputs)

prof.export_chrome_trace("spellchecker/kosenko/sandbox/profiling/2_pytorch_profiler_recipe/trace.json")

You can examine the sequence of profiled operators and CUDA kernels
in Chrome trace viewer (``chrome://tracing``):

<img src="file://../../_static/img/trace_img.png" scale="25 %">



In [10]:
# chrome://tracing

### 6. Examining stack traces

Profiler can be used to analyze Python and TorchScript stack traces:



In [11]:
with profile(
    activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
    with_stack=True,
) as prof:
    model(inputs)

# Print aggregated stats
print(prof.key_averages(group_by_stack_n=5).table(sort_by="self_cuda_time_total", row_limit=2))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                aten::cudnn_convolution        23.71%     924.088us        35.44%       1.381ms      69.063us       3.055ms        74.69%       3.055ms     152.770us            20  
void cudnn::engines_precompiled::nchwToNhwcKernel<fl...         0.00%       0.000us         0.00%       0.000us       0.000us     558.300us        13.65%     558.300us      16.421us            34  
---------

Note the two convolutions and the two call sites in ``torchvision/models/resnet.py`` script.

(Warning: stack tracing adds an extra profiling overhead.)



### 7. Visualizing data as a flame graph

Execution time (``self_cpu_time_total`` and ``self_cuda_time_total`` metrics) and stack traces
can also be visualized as a flame graph. To do this, first export the raw data using ``export_stacks`` (requires ``with_stack=True``):



In [12]:
prof.export_stacks("/tmp/profiler_stacks.txt", "self_cuda_time_total")

We recommend using [Flamegraph tool](https://github.com/brendangregg/FlameGraph) to generate an
interactive ``.svg`` file:

```sh
git clone https://github.com/brendangregg/FlameGraph
cd FlameGraph
./flamegraph.pl --title "CUDA time" --countname "us." /tmp/profiler_stacks.txt > perf_viz.svg
```


<img src="file://../../_static/img/perf_viz.png" scale="25 %">



### 8. Using profiler to analyze long-running jobs

PyTorch profiler offers an additional API to handle long-running jobs
(such as training loops). Tracing all of the execution can be
slow and result in very large trace files. To avoid this, use optional
arguments:

- ``schedule`` - specifies a function that takes an integer argument (step number)
  as an input and returns an action for the profiler, the best way to use this parameter
  is to use ``torch.profiler.schedule`` helper function that can generate a schedule for you;
- ``on_trace_ready`` - specifies a function that takes a reference to the profiler as
  an input and is called by the profiler each time the new trace is ready.

To illustrate how the API works, let's first consider the following example with
``torch.profiler.schedule`` helper function:



In [13]:
from torch.profiler import schedule

my_schedule = schedule(
    skip_first=10,
    wait=5,
    warmup=1,
    active=3,
    repeat=2)

Profiler assumes that the long-running job is composed of steps, numbered
starting from zero. The example above defines the following sequence of actions
for the profiler:

1. Parameter ``skip_first`` tells profiler that it should ignore the first 10 steps
   (default value of ``skip_first`` is zero);
2. After the first ``skip_first`` steps, profiler starts executing profiler cycles;
3. Each cycle consists of three phases:

   - idling (``wait=5`` steps), during this phase profiler is not active;
   - warming up (``warmup=1`` steps), during this phase profiler starts tracing, but
     the results are discarded; this phase is used to discard the samples obtained by
     the profiler at the beginning of the trace since they are usually skewed by an extra
     overhead;
   - active tracing (``active=3`` steps), during this phase profiler traces and records data;
4. An optional ``repeat`` parameter specifies an upper bound on the number of cycles.
   By default (zero value), profiler will execute cycles as long as the job runs.



Thus, in the example above, profiler will skip the first 15 steps, spend the next step on the warm up,
actively record the next 3 steps, skip another 5 steps, spend the next step on the warm up, actively
record another 3 steps. Since the ``repeat=2`` parameter value is specified, the profiler will stop
the recording after the first two cycles.

At the end of each cycle profiler calls the specified ``on_trace_ready`` function and passes itself as
an argument. This function is used to process the new trace - either by obtaining the table output or
by saving the output on disk as a trace file.

To send the signal to the profiler that the next step has started, call ``prof.step()`` function.
The current profiler step is stored in ``prof.step_num``.

The following example shows how to use all of the concepts above:



In [14]:
def trace_handler(p):
    output = p.key_averages().table(sort_by="self_cuda_time_total", row_limit=10)
    print(output)
    temp = 'spellchecker/kosenko/sandbox/profiling/2_pytorch_profiler_recipe'
    p.export_chrome_trace(
        f"{temp}/trace_" + str(p.step_num) + ".json"
    )

with profile(
    activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
    schedule=torch.profiler.schedule(
        wait=1,
        warmup=1,
        active=2),
    on_trace_ready=trace_handler
) as p:
    for idx in range(8):
        model(inputs)
        p.step()

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                          ProfilerStep*         0.00%       0.000us         0.00%       0.000us       0.000us       8.384ms        47.18%       8.384ms       4.192ms             2  
                                aten::cudnn_convolution        10.13%     965.568us        16.67%       1.588ms      39.709us       6.114ms        34.41%       6.114ms     152.861us            40  
sm86_xmma

## Learn More

Take a look at the following recipes/tutorials to continue your learning:

-  [PyTorch Benchmark](https://pytorch.org/tutorials/recipes/recipes/benchmark.html)
-  [PyTorch Profiler with TensorBoard](https://pytorch.org/tutorials/intermediate/tensorboard_profiler_tutorial.html) tutorial
-  [Visualizing models, data, and training with TensorBoard](https://pytorch.org/tutorials/intermediate/tensorboard_tutorial.html) tutorial


